In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange
import tensorflow as tf

In [3]:
vocabulary_size = 50000

# reading data
file = open("text8", "r") 
vocabulary = tf.compat.as_str(file.read()).split()
print('Data size', len(vocabulary))

# most common words
count = [['UNK', -1]]
count.extend(collections.Counter(vocabulary).most_common(vocabulary_size - 1))

# embeddings
embed = dict()
for word, _ in count:
    embed[word] = len(embed)

rev_embed = dict(zip(embed.values(), embed.keys()))

# generating embedded data
data = list()
unk = 0
for word in vocabulary:
    if word in embed:
        index = embed[word]
    else:
        index = 0    # embed['UNK']
        unk += 1
    data.append(index)
count[0][1] = unk

# to reduce memory
del vocabulary

print('Most common words:', count[:10])
print('Sample data:\n', data[:10],'\n', [rev_embed[i] for i in data[:10]])

Data size 17005207
Most common words: [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764), ('in', 372201), ('a', 325873), ('to', 316376), ('zero', 264975), ('nine', 250430)]
Sample data:
 [5241, 3083, 12, 6, 195, 2, 3134, 46, 59, 156] 
 ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']


In [4]:
# generate a training batch
data_index = 0

def generate_batch( batch_size, num_skips):
    global data_index
    skip_window = num_skips//2
    assert batch_size % num_skips == 0
    
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1    # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window    # target label at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
        
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2)
print("Example Batch: (input word -> output context)")
for i in range(len(batch)):
    print(batch[i], rev_embed[batch[i]],'->', labels[i, 0], rev_embed[labels[i, 0]])
data_index = 0

Example Batch: (input word -> output context)
3083 originated -> 5241 anarchism
3083 originated -> 12 as
12 as -> 6 a
12 as -> 3083 originated
6 a -> 12 as
6 a -> 195 term
195 term -> 6 a
195 term -> 2 of


In [5]:
# Step 4: Build and train a skip-gram model.

batch_size = 128
embed_size = 128    # Dimension of the embedding vector.
num_skips = 2       # How many times to reuse an input to generate a label.


# Validation samples: most frequent words
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()
with graph.as_default():

    # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Look up embeddings for inputs.
    embeddings = tf.Variable( tf.random_uniform( [vocabulary_size, embed_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup( embeddings, train_inputs)

    # Construct the variables for the NCE loss
    W = tf.Variable( tf.truncated_normal( [vocabulary_size, embed_size], stddev=1.0/math.sqrt(embed_size)))
    b = tf.Variable( tf.zeros( [vocabulary_size]))

    # Compute the average NCE loss for the batch.
    # automatically draws a new sample of the nega labels each time we eval the loss.
    loss = tf.reduce_mean( tf.nn.nce_loss(weights=W, biases=b, labels=train_labels, 
                                          inputs=embed, num_sampled=num_sampled, num_classes=vocabulary_size))

    # Construct the SGD optimizer using a learning rate of 1.0.      
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt( tf.reduce_sum( tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup( normalized_embeddings, valid_dataset)
    similarity = tf.matmul( valid_embeddings, normalized_embeddings, transpose_b=True)

    # Add variable initializer.
    init = tf.global_variables_initializer()

In [6]:
# Step 5: Begin training.
num_steps = 100001

with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()
    print('Initialized')

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch( batch_size, num_skips)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # estimated loss over the last 2000 batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange( valid_size):
                valid_word = rev_embed[valid_examples[i]]
                top_k = 4    # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
                for k in xrange(top_k):
                    close_word = rev_embed[nearest[k]]
                    log_str = '%s %s,' % (log_str, close_word)
                print(log_str)
            print("")
                
    final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  295.685028076
Nearest to may: columbo, shatter, tropics, lenat,
Nearest to years: luthor, bund, sings, induces,
Nearest to was: trademarks, gibson, anglian, genji,
Nearest to UNK: affleck, drexel, glove, surgically,
Nearest to or: rimet, hengest, swordfish, octavio,
Nearest to as: stara, sesotho, stouffer, birthrate,
Nearest to a: coordinated, porting, rutger, ishtar,
Nearest to for: distraction, zubaydah, oyly, sarris,
Nearest to system: remake, lookups, norm, marta,
Nearest to its: rcito, oc, jackal, lir,
Nearest to these: attainder, catacombs, saturday, unalaska,
Nearest to up: defamation, psychopathic, foundry, steward,
Nearest to see: bagpipe, prescribe, bobble, blatant,
Nearest to six: interpolant, gra, resonant, shiloh,
Nearest to while: ken, jura, soap, rancho,
Nearest to are: cumberland, bosman, detailing, baskets,

Average loss at step  2000 :  113.67349466
Average loss at step  4000 :  52.040497231
Average loss at step  6000 :  33.33521

Average loss at step  82000 :  4.77486974752
Average loss at step  84000 :  4.74932595718
Average loss at step  86000 :  4.77752174807
Average loss at step  88000 :  4.74899364591
Average loss at step  90000 :  4.73518619788
Nearest to may: can, would, will, could,
Nearest to years: architecture, cardiomyopathy, cl, luthor,
Nearest to was: is, had, were, has,
Nearest to UNK: kapoor, microcebus, dasyprocta, ursus,
Nearest to or: and, kapoor, cardiomyopathy, michelob,
Nearest to as: hominem, leontopithecus, michelob, dasyprocta,
Nearest to a: the, any, pulau, agouti,
Nearest to for: primigenius, michelob, busan, kapoor,
Nearest to system: dinar, aba, lookups, schuschnigg,
Nearest to its: their, his, the, thaler,
Nearest to these: many, some, such, both,
Nearest to up: steward, off, microcebus, them,
Nearest to see: olden, franchisee, pulau, called,
Nearest to six: eight, seven, five, four,
Nearest to while: although, but, and, escuela,
Nearest to are: were, is, have, be,

Average loss at

In [8]:
# Step 6: Visualize the embeddings.

def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
    assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
    plt.figure(figsize=(18, 18))    # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points', ha='right', va='bottom')
    plt.savefig(filename)

# pylint: disable=g-import-not-at-top
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
plot_only = 500
low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
labels = [rev_embed[i] for i in xrange(plot_only)]
plot_with_labels(low_dim_embs, labels)


a a a a a a a a a a a a a a a a a 

IndexError: index 16 is out of bounds for axis 0 with size 16